# Tutorial "ELECTRE_Tri_B_main.py"

Author: [Souleymane Daniel](mailto:souleymane.daniel@insa-lyon.fr)

[INSA Lyon](https://www.insa-lyon.fr), Lyon, France, 17/03/2022

## Introduction

In order to dissociate the representative functions of the ELECTRE Tri-B method and their execution in the case of examples or concrete applications, a "main" executable code has been created. The [**ELECTRE_Tri_B_main.py**](ELECTRE_Tri_B_main.py) code is the executable. It contains the different instructions that will lead to the construction of the objects used throughout the method. The different stages of this executable code are presented here.

In [1]:
import ELECTRE_Tri_B
import pandas as pd
import math

## 1. First step : Ranking categories

The first step is to define the names of the different ranking categories, which should be given in ascending order from worst to best in the form of a Python list. Be careful that this list of classification categories **{C(1);...;C(k);...;C(q)}** is consistent with the boundary reference actions that delimit them **{b(0);...;b(k);...;b(q)}**.

In [2]:
Categories = ['C1', 'C2', 'C3', 'C4', 'C5']

## 2. Second step : Importing the input data

The second step consist of importing the input data of the problem. These data should be stored in csv files according to the data structure presented in the document [Tutorial_CSV_files_format.md](Tutorial_CSV_files_format.md). There should be four such files:
- a csv file containing the different data related to the criteria and their weight.
    *Example: [1.Weights.csv](1.Weights.csv)*
- a csv file containing the different data related to the actions and their performances.
    *Example: [2.Actions_performances.csv](2.Actions_performances.csv)*
- a csv file containing the different data related to the boundary reference actions and their performances. 
    *Example: [3.Boundaries_actions_performances.csv](3.Boundaries_actions_performances.csv)*
- a csv file containing the different data related to the thresholds. 
    *Example: [4.Thresholds.csv](4.Thresholds.csv)*

To import the data, the following function is used, where the input parameters are of the csv files names:
- ***input_data(name_W, name_AP, name_BP, name_T)***

This functions will then return the different objects needed for the rest of the method.

In [3]:
C, W, A, AP, B, BP, T = ELECTRE_Tri_B.input_data('1.Weights.csv',
                                                 '2.Actions_performances.csv',
                                                 '3.Boundaries_actions_performances.csv',
                                                 '4.Thresholds.csv')

In [4]:
print("Criteria:", C)
print('----------------')
print("Weights:")
print(pd.DataFrame.from_dict(W, orient='index', columns=['W']).round(2))
print('----------------')
print("Actions:", A)
print('----------------')
print("Actions performances:")
print(pd.DataFrame(AP).round(1))
print('----------------')
print("Boundaries actions:", B)
print('----------------')
print("Boundaries actions performances:")
print(pd.DataFrame(BP).round(1))
print('----------------')
print("Thresholds:")
print(pd.DataFrame(T, index=['qi', 'pi', 'vi']).round(2))

Criteria: ['g1.1', 'g1.2', 'g1.3', 'g1.4', 'g1.5', 'g2.1', 'g2.2', 'g2.3', 'g2.4', 'g3.1', 'g3.2', 'g3.3', 'g3.4', 'g4.1', 'g4.2', 'g4.3']
----------------
Weights:
          W
g1.1  11.56
g1.2  10.11
g1.3   4.33
g1.4  11.56
g1.5   5.78
g2.1   6.48
g2.2   9.07
g2.3   3.89
g2.4   3.89
g3.1   6.37
g3.2   5.39
g3.3   2.45
g3.4   2.45
g4.1   7.78
g4.2   5.56
g4.3   3.33
----------------
Actions: ['S1.1', 'S1.2', 'S1.3', 'S1.4', 'S2.1', 'S2.2', 'S2.3', 'S2.4', 'S3.1', 'S3.2', 'S3.3', 'S3.4', 'S4.1', 'S4.2', 'S4.3', 'S4.4', 'S5.1', 'S5.2', 'S5.3', 'S5.4', 'S6.1', 'S6.2', 'S6.3', 'S6.4', 'S7.1', 'S7.2', 'S7.3', 'S7.4']
----------------
Actions performances:
           S1.1       S1.2       S1.3      S1.4       S2.1       S2.2  \
g1.1       -0.0 -1008654.5 -1260873.8 -905164.5 -1239757.1 -1397246.8   
g1.2 -1757134.0  -551661.0  -711361.0 -551661.0  -551661.0  -625566.0   
g1.3        0.0   952088.4   991697.2  917604.3   955885.1   996167.8   
g1.4   -82701.2   -41345.8   -36119.3  -42267.8  

## 3. Third step : Cutting threshold

The third step is to define the "**cutting threshold λ**". The cutting threshold is the basis of the comparison. It allows deciding on the existing over-ranking relationships between an actions "**_a(i)_**" and a boundary reference action "**_b(k)_**". The closer it is to 1, the more demanding the classification will be and may lead to situations of incomparability. The most common values for this cutting threshold are generally between **0.50** and **0.75**.

It is interesting to calculate the degree of separability between two consecutive reference actions "**_σ(bk,bk+1)_**" in order to determine with which level of difference the performance categories are constructed. This degree of separability is equal to the credibility of the statement **_b(k)_** outperforms **_b(k+1)_**. The ***separability_test(C, W, B, BP, T, display='NO')*** function can be used to perform this calculation. The maximum value of the credibilities calculated in this way corresponds to the minimum required credibility threshold **_λ_**.

In [5]:
Sigma_bk_init, Separability_init = ELECTRE_Tri_B.separability_test(C, W, B, BP, T, display='YES')
λ = 0.50
print('Chosen initial credibility threshold λ =', λ)

 
The degree of separability is Hyper-strict
Minimum required credibility threshold : max(σ(bk, bk+1)) = 0.0
Chosen initial credibility threshold λ = 0.5


## 4. Fourth step : Automatic execution of the ELECTRE Tri-B

In the fourth step the objective is to use the input data to calculate the indicators of the ELECTRE Tri-B method. This calculation can be done automatically using the **ELECTRE_Tri_B(C, W, A, AP, B, BP, T, CAT, λ, display='YES')** function. This function automatically executes the different calculation steps of the ELECTRE Tri-B method in order to obtain the final rankings of the alternatives within the different categories.

In [6]:
RESULTS = ELECTRE_Tri_B.ELECTRE_Tri_B(C, W, A, AP, B, BP, T, Categories, λ, display='YES')

 
Results of the pessimistic sorting : 
C1 : ['S1.1', 'S5.1', 'S5.2', 'S5.3', 'S5.4', 'S7.1', 'S7.2', 'S7.3', 'S7.4']
C2 : ['S1.2', 'S1.4', 'S2.1', 'S3.1', 'S4.1', 'S4.2', 'S4.3', 'S4.4', 'S6.3']
C3 : ['S1.3', 'S3.2', 'S3.3', 'S3.4', 'S6.1', 'S6.2', 'S6.4']
C4 : ['S2.2', 'S2.3', 'S2.4']
C5 : []
Pessimistic category : {'S1.1': 1, 'S1.2': 2, 'S1.3': 3, 'S1.4': 2, 'S2.1': 2, 'S2.2': 4, 'S2.3': 4, 'S2.4': 4, 'S3.1': 2, 'S3.2': 3, 'S3.3': 3, 'S3.4': 3, 'S4.1': 2, 'S4.2': 2, 'S4.3': 2, 'S4.4': 2, 'S5.1': 1, 'S5.2': 1, 'S5.3': 1, 'S5.4': 1, 'S6.1': 3, 'S6.2': 3, 'S6.3': 2, 'S6.4': 3, 'S7.1': 1, 'S7.2': 1, 'S7.3': 1, 'S7.4': 1}
 
Results of the optimistic sorting : 
C1 : []
C2 : ['S1.1', 'S5.1', 'S5.2', 'S5.3', 'S5.4', 'S7.1', 'S7.2', 'S7.3', 'S7.4']
C3 : ['S1.2', 'S1.4', 'S2.1', 'S3.1', 'S4.1', 'S4.2', 'S4.3', 'S4.4', 'S6.3']
C4 : ['S1.3', 'S2.2', 'S2.3', 'S2.4', 'S3.2', 'S3.3', 'S3.4', 'S6.1', 'S6.2', 'S6.4']
C5 : []
Optimistic category :  {'S1.1': 2, 'S1.2': 3, 'S1.3': 4, 'S1.4': 3, 'S2.1':

### 4.1 Concordance matrices
This is an indicator of how well an action ***a(i)*** is at least as good as the reference profil ***b(k)*** for a given criterion ***g(j)***.

In [7]:
print('Concordance matrix for the "b0" boundary reference profile: c(ai,b0)')
pd.DataFrame(RESULTS[0]['b0']['c(ai,b0)'], index=A, columns=C).round(2)

Concordance matrix for the "b0" boundary reference profile: c(ai,b0)


,g1.1,g1.2,g1.3,g1.4,g1.5,g2.1,g2.2,g2.3,g2.4,g3.1,g3.2,g3.3,g3.4,g4.1,g4.2,g4.3
S1.1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
S1.2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
S1.3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
S1.4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
S2.1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
S2.2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
S2.3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
S2.4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
S3.1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
S3.2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


### 4.2 Discordance matrices
This indicator is expressed using the veto threshold. They mark the limit beyond which the hypothesis that a given action ***a(i)*** outperforms a boundary reference action ***b(k)*** for a given criterion ***g(j)*** can be rejected without affecting the credibility of the opposite hypothesis.

In [8]:
print('Discordance matrix for the "b0" boundary reference profile: d(ai,b0)')
pd.DataFrame(RESULTS[1]['b0']['d(ai,b0)'], index=A, columns=C).round(2)

Discordance matrix for the "b0" boundary reference profile: d(ai,b0)


,g1.1,g1.2,g1.3,g1.4,g1.5,g2.1,g2.2,g2.3,g2.4,g3.1,g3.2,g3.3,g3.4,g4.1,g4.2,g4.3
S1.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S1.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S1.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S2.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S2.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S2.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S2.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S3.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 4.3 Global concordances vectors
The global concordance indices allow stating to what extent the hypothesis "the action ***a(i)*** globally outperforms the boundary reference action ***b(k)***" is met.

In [9]:
print('Global concordances vectors for the "b0" boundary reference profile: C(ai,b0) and C(b0,ai)')
pd.DataFrame(RESULTS[2]['b0'], index=A).round(2)

Global concordances vectors for the "b0" boundary reference profile: C(ai,b0) and C(b0,ai)


,"C(ai,b0)","C(b0,ai)"
S1.1,1.0,0.00
S1.2,1.0,0.00
S1.3,1.0,0.00
S1.4,1.0,0.00
S2.1,1.0,0.00
S2.2,1.0,0.00
S2.3,1.0,0.00
S2.4,1.0,0.00
S3.1,1.0,0.00
S3.2,1.0,0.00


### 4.4 Credibility vectors
The credibility of the outranking relationships between the actions and the boundary reference profile varies from pair to pair and is represented by the degree of credibility of the outranking.

In [10]:
print('Credibility vectors for the "b0" boundary reference profile: σ(ai,b0) and σ(b0,ai)')
pd.DataFrame(RESULTS[3]['b0'], index=A).round(2)

Credibility vectors for the "b0" boundary reference profile: σ(ai,b0) and σ(b0,ai)


,"σ(ai,b0)","σ(b0,ai)"
S1.1,1.0,0.0
S1.2,1.0,0.0
S1.3,1.0,0.0
S1.4,1.0,0.0
S2.1,1.0,0.0
S2.2,1.0,0.0
S2.3,1.0,0.0
S2.4,1.0,0.0
S3.1,1.0,0.0
S3.2,1.0,0.0


### 4.5
Matrix of outranking relations
The lambda cutting threshold value is compare to the credibilities values to decide on four over ranking relationships:
- preference of *a(i)* over *b(k)*: "**>**"
- preference of *b(k)* over *a(i)*: "**<**"
- indifference: "**I**"
- incomparability: "**R**"

In [11]:
pd.DataFrame(RESULTS[4], index=A)

,b0,b1,b2,b3,b4,b5
S1.1,>,R,R,R,R,<
S1.2,>,>,R,R,R,<
S1.3,>,>,>,R,R,<
S1.4,>,>,R,R,R,<
S2.1,>,>,R,R,<,<
S2.2,>,>,>,>,<,<
S2.3,>,>,>,>,<,<
S2.4,>,>,>,>,<,<
S3.1,>,>,R,R,<,<
S3.2,>,>,>,R,<,<


### 4.6 Pessimistic and Optimistic ranking procedure
Two sorting procedures are performed based on the previous over-ranking relationships. Each of these sorting procedures assigns actions to a specific performance category. The difference between the two procedures is the ranking of incomparabilities (***R***).

In [12]:
print('Results of the pessimistic sorting : ')
for cat in Categories:
    print('{}:'.format(cat), RESULTS[5][0][cat])

Results of the pessimistic sorting : 
C1: ['S1.1', 'S5.1', 'S5.2', 'S5.3', 'S5.4', 'S7.1', 'S7.2', 'S7.3', 'S7.4']
C2: ['S1.2', 'S1.4', 'S2.1', 'S3.1', 'S4.1', 'S4.2', 'S4.3', 'S4.4', 'S6.3']
C3: ['S1.3', 'S3.2', 'S3.3', 'S3.4', 'S6.1', 'S6.2', 'S6.4']
C4: ['S2.2', 'S2.3', 'S2.4']
C5: []


In [13]:
print('Results of the optimistic sorting : ')
for cat in Categories:
    print('{}:'.format(cat), RESULTS[6][0][cat])

Results of the optimistic sorting : 
C1: []
C2: ['S1.1', 'S5.1', 'S5.2', 'S5.3', 'S5.4', 'S7.1', 'S7.2', 'S7.3', 'S7.4']
C3: ['S1.2', 'S1.4', 'S2.1', 'S3.1', 'S4.1', 'S4.2', 'S4.3', 'S4.4', 'S6.3']
C4: ['S1.3', 'S2.2', 'S2.3', 'S2.4', 'S3.2', 'S3.3', 'S3.4', 'S6.1', 'S6.2', 'S6.4']
C5: []


### 4.7 Median rank
When the two sorting procedures do not lead to the same results, a median rank is calculated. So an action classified as "**_C2_**" by the optimistic sorting and "**_C1_**" by the pessimistic sorting, will therefore belong to the "**_C21_**" category with a median rank of **1.5** (it will be less preferable than an action belonging to the "**_C22_**" category with a median rank of **2.0**).

In [16]:
pd.DataFrame.from_dict(RESULTS[7], orient='index', columns=['Median rank'])

,Median rank
S1.1,1.5
S1.2,2.5
S1.3,3.5
S1.4,2.5
S2.1,2.5
S2.2,4.0
S2.3,4.0
S2.4,4.0
S3.1,2.5
S3.2,3.5
